### Tensorflow Problem Sheet
In this problem sheet I will be using keras with tensorflow to predict the species of Iris from a flowers sepal length and width and a petals length and width.

The aim of this problem sheet is to get a better understanding of how tensorflow works.

As mentioned above, I'm using Fisher's Iris Dataset. If you're looking for a little bit of further reading about the Iris dataset, check out my other notebook [here](https://github.com/ImErvin/JupyterPyplotNumpy-Problem-Sheet/blob/master/IrisNotebook.ipynb).

**Note:** The code is written by [salmanahmad4u](https://github.com/salmanahmad4u/keras-iris/blob/master/iris_nn.py) and adapted by [Ian Mcloughlin](https://github.com/ianmcloughlin). The minor adaptation, explanation and analysis of the code is provided by me, [Ervin Mamutov](https://github.com/imervin).

### What is Tensorflow and Keras?

Tensorflow is a popular software library for dataflow programming across a range of tasks. Tensorflow is open-source and is developed by the Google Brain Team. Tensorflow is a symbolic math library and is also used for machine learning applications such as neaural networks [1]. I will be using Tensorflow's Python API but it is available for a range of languages.

Keras is an open source neural network library written in Python developed by a Google engineer: Francois Chollet. Keras acts like a "library on top of a library" as it is capable of running on top of MXNet, Deeplearning4j, Tensorflow, CNTK or Theano. Keras takes the functionality in core Tensorflow and adds a higher-level of abstraction to it, making it easier to experiment with deep neaural networks [2].


### Create the Tensorflow model

I'm using Keras so instead of importing tensorflow, I can import Keras which uses tensorflow as the backend.
I also import additional useful libraries such as numpy for dealing with complicated arrays and csv to read the iris csv dataset.

In [1]:
import numpy as np
import keras as kr
import csv

Using TensorFlow backend.


The iris dataset contains 150 rows of data, the dataset I'm using is ordered. The first 50 are setosa, the next 50 are versicolor and the last 50 are virginica. Each row contains 5 different pieces of information about the flower: the sepal length, the sepal width, the petal length, the petal width and finally the iris class (e.g: setosa, virginica etc.)

I can use the 'csv' library to read in the iris dataset and to store it into relevant numpy arrays to later use that data to create the model.

In [2]:
# Initiate iris as a list with the conents of IRIS_dataset.csv line by line starting on the first line ([0:])
iris = list(csv.reader(open('IRIS_dataset.csv')))[0:]

# Expected to be 150
print("Length of the list 'iris':",len(iris))

# Expected to be of class versicolor and first 4 indexs to be float variables.
print("51st element of list 'iris':",iris[50])

Length of the list 'iris': 150
51st element of list 'iris': ['7.0', '3.2', '4.7', '1.4', 'Iris-versicolor']


Now that the iris dataset has been loaded into an array successfully, I will split the data into input and outputs.

From looking at the data above, the first 4 floats look like they should be the input because they make up the class of Iris. If the first 4 elements are the inputs then the 5th element is the output (class of iris).

I can use numpy to create an array of inputs and outputs.

In [3]:
# Initiate inputs as a numpy array that's a subset of iris - reading the first 4 indices
# as floats representing the sepal length/width and petal length/width
# [:,:4] is numoy notation for reading a 2D array and splicing it.
# It means: "Take all rows of iris, within each row, return the first 4 indices as floats"
inputs = np.array(iris)[:,:4].astype(np.float)

# Initiate outputs as a numpy array that's a subset of iris - reading the last index
# representing the iris class.
# [:,4:] means: "Take all rows, and within each row splice the first 4 indices and return the remaining"
outputs = np.array(iris)[:,4:]

# Expected to be [7.0, 3.2, 4.7, 1.4] & Iris-versicolor - same as the output of the 51st element above.
print("51st element of inputs & outputs:",inputs[50],"&",outputs[50])

51st element of inputs & outputs: [ 7.   3.2  4.7  1.4] & ['Iris-versicolor']


The expected values and actual values above are matching meaning that I have successfully separated the iris list into inputs and outputs arrays.

Now the outputs array needs some work because it's basically an array of recurring strings i.e: "Iris-setosa" x 50, "Iris-versicolor" x 50 etc. The main problem with this is that we're working with strings and to use libraries like keras and tensorflow we need to use integers.

A better representation would be to split this array into two  - one for the unique string that occurs e.g: ["Iris-setosa","Iris-versicolor","Iris-virginica"] and another for the index of where that number occurs e.g: Iris-seotsa occurs between in the range of 0..49 etc.

Numpy's '.unique' function allows to do exactly that.

In [4]:
# Explanation found on - https://docs.scipy.org/doc/numpy/reference/generated/numpy.unique.html
# Initiate output_vals to the unique values that occur in 'outputs'
# Initiate outputs_ints to an array of the indices where the unique values sit.
outputs_vals, outputs_ints = np.unique(outputs, return_inverse=True)

# Expected to print out ["Iris-setosa" "Iris-versicolor" "Iris-virginica"] 
print("Unique values in the array 'outputs':",outputs_vals)
# Expected to print out something like [0 0 0 0 0 .. 1 1 1 1 1 .. 2 2 2 2 2 ..]
print("Where the unique values occur in the array 'outputs':\n%s"%(outputs_ints))
# Expected to be Iris-versicolor
print("Class of iris that sits on the 51st index of outputs_ints:",outputs_vals[outputs_ints[50]])

Unique values in the array 'outputs': ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
Where the unique values occur in the array 'outputs':
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Class of iris that sits on the 51st index of outputs_ints: Iris-versicolor


Now that I have split the array into unique values and index occurrences I can start preparing for categorical cross entropy..

### What is cross entropy?
Cross-entropy is commonly used to quantify the difference between two probability distributions. Usually the "true" distribution (the one that your machine learning algorithm is trying to match) is expressed in terms of a one-hot distribution [3].

#### What is a multi-class classification?
A multi-class classification is a classification task with more than two classes; e.g., classify a set of images of fruits which may be oranges, apples, or pears. Multiclass classification makes the assumption that each sample is assigned to one and only one label: a fruit can be either an apple or a pear but not both at the same time [4].

#### What is a multi-label classification?
A multi-label classification assigns to each sample a set of target labels. This can be thought as predicting properties of a data-point that are not mutually exclusive, such as topics that are relevant for a document. A text might be about any of religion, politics, finance or education at the same time or none of these [4].

#### Why does this matter?
I could use a categorical or binary cross entropy - categorical tackles a multi-class problem while binary tackles a multi-label problem. The Iris problem is a multi-class problem (An Iris can either be setosa, virginica or versicolor), hence why I need to use categorical cross entropy when calculating the loss.

To prepare I need to take the 'outputs_int' and turn it into a binary matrix of "categories" that will look something like: [1. 0. 0.], [0. 1. 0.], [0. 0. 1.]. This will later be used for the loss function.

Keras comes with a utility that does exactly that. The utils.to_categorical utility converts a class vector to binary class matrix [5].


In [5]:
# Initiate outputs_cats to an array of categories setting the number of classes to 3 
outputs_cats = kr.utils.to_categorical(outputs_ints,  num_classes=3)

# Expected to be [0. 1. 0.] 
print("The 51st element of outputs_cats:", outputs_cats[50])

The 51st element of outputs_cats: [ 0.  1.  0.]


### Split the data into training and testing

Tensorflow works by training and testing a model. The iris dataset doesn't have a specific training set and testing set so I'll need to create my own training and testing set. I can achieve this by splitting the dataset in two and using the first half as the training set and using the other as the testing set. The problem with this is that the data is sorted.

To overcome this problem I'll need to randomly shuffle the dataset indices and split the result in to two arrays.

I will use Numpy's .random.permutation, which randomly permute a sequence, or return a permuted range [6] and Numpy's .array_split lets me split an array into multiple sub-arrays [7]. I could use Numpy's random.shuffle but I would not be able to keep track of the indicies.

In [6]:
# Initiate indices to be a random shuffle of 0 to 149 (length of inputs) resulting in something like [3 132 56 44 ..]
indices = np.random.permutation(len(inputs))

# Initiate train_indices to the first 75 values of indices and test_indices to the other 75 values of indices.
train_indices, test_indices = np.array_split(indices, 2)

print("75th index of array 'indices':", indices[74])
print("76th index of array 'indices':", indices[75])
# Expected to be the same as the 75th index of indices
print("First element of the test_indices:", train_indices[74])
# Expected to be the same as the 76th index of indices
print("Last element of the train_indices:", test_indices[0])
print("Value at the",indices[74],"(aka the 75th element of 'indices') index of 'inputs':",inputs[indices[74]])
print("Value at the",indices[75],"(aka the 76th element of 'indices') index of 'inputs':",inputs[indices[75]])
print()
# Initiate inputs_train to the values of inputs at the indices of train_indices
# Initiate output_train to the values of outputs_cats at the indicies of train_indices
# E.g: If train_indices is [2,5,10], it will set inputs_train to the values found at inputs[2],inputs[5] and inputs[10]
# and likewise for outputs_train
inputs_train, outputs_train = inputs[train_indices], outputs_cats[train_indices]

# Do the same as above for the testing set.
inputs_test,  outputs_test  = inputs[test_indices],  outputs_cats[test_indices]

# Expect the value of last element of inputs_train should be the same as the value of the 
# random index (produced in indices) of inputs
print("Last element of inputs_train:", inputs_train[74])
print("The",indices[74],"index of inputs:", inputs[indices[74]])
# Same expectation for the other prints below.
print("Last element of outputs_train:", outputs_train[74])
print("The",indices[74],"index of outputs_cats:", outputs_cats[indices[74]])
print()
print("First element of inputs_test:", inputs_test[0])
print("The",indices[75],"index of inputs:", inputs[indices[75]])
print("Last element of outputs_train:", outputs_test[0])
print("The",indices[75],"index of outputs_cats:", outputs_cats[indices[75]])

75th index of array 'indices': 85
76th index of array 'indices': 143
First element of the test_indices: 85
Last element of the train_indices: 143
Value at the 85 (aka the 75th element of 'indices') index of 'inputs': [ 6.   3.4  4.5  1.6]
Value at the 143 (aka the 76th element of 'indices') index of 'inputs': [ 6.8  3.2  5.9  2.3]

Last element of inputs_train: [ 6.   3.4  4.5  1.6]
The 85 index of inputs: [ 6.   3.4  4.5  1.6]
Last element of outputs_train: [ 0.  1.  0.]
The 85 index of outputs_cats: [ 0.  1.  0.]

First element of inputs_test: [ 6.8  3.2  5.9  2.3]
The 143 index of inputs: [ 6.8  3.2  5.9  2.3]
Last element of outputs_train: [ 0.  0.  1.]
The 143 index of outputs_cats: [ 0.  0.  1.]


The above outputs indicate that I shuffled the indices of inputs succesfully and split the data into training and testing sets. This is a very common method of splitting datasets into a training and testing set, using a mask of indices rather than using the actual dataset preserves the dataset and still allows me to reflect the data on the original dataset.

Now that I have split my dataset into training and testing sets, the data is ready to be processed in a neural network model.

### What is a Model?
There are many definitions of a model in different contexts. In the context of machine learning (ML), a model is an alias for a trained (or yet to be trained) model which is expected to perform some intelligent stuff. For eg. A chat bot is an ML model which may comprise of a Neural Net to interpret speech and convert it into text and another statistical model to filter the keywords of the converted speech query [8].

In Tensorflow a model is represented as a dataflow graph it can be something as simple as an add function e.g: tensors a and b produce output c, model that sits inbetween is M which adds a and b.

The model I'm using will contain a stack of layers. Keras comes with a "Sequential" model which is a linear stack of layers. The syntax for this is simply keras.models.Sequential().

In [32]:
# Initiate model as a sequential keras model.
model = kr.models.Sequential()

# Expected to be 0
print("Number of layers in my model:", len(model.layers))

Number of layers in my model: 0


Now that my model exists, I need to add layers to it.

There are Dense and Activation layers.

#### What is a dense and activation layer?
A dense layer is just your regular densely-connected NN layer. This is where all the nodes live be it input/output or hidden. An activation layer applies an activation function to an output [9]. There are several activation formulas such as Sigmoiad, softmax, relu etc.

#### Why would we use an activation function??
Activation functions introduce non-linear properties to the network. This is important because linear functions(2x) can only do so much while non-linear functions(2x^2) offer a much more complex and realistic output [10]. 

I will be using relu on my hidden layers and softmax on my output layers.

Rectified Linear Unit (Relu) function is the most popular activation function at the moment because it doesn't involve expensive operations like TanH or Sigmoid (two other hidden layer activation formulas) so it learns faster and also avoid the vanishing gradient problem [10].

Softmax is used to find the probability of multi-class classifications (as mentioned above). This activation formula will return the probabilty of which Iris class the test data is most closely relatable to.

In [41]:
model = kr.models.Sequential([
    # Define input_shape as 4 (4 input nodes : sepal l/w and petal l/w) and create 12 hidden layer nodes
    kr.layers.Dense(12, input_shape=(4,)),
    # Set the activation formula to relu on the dense layer above
    kr.layers.Activation("relu"),
    # Connect another dense layer of 3 output nodes (for each Iris class). As this is the last layer it is the output layer.
    kr.layers.Dense(3),
    # Set the activation formula to softmax on the output dense layer
    kr.layers.Activation("softmax")
])

# Expected to be 4 as I added 4 above.
print("Number of layers in my model:", len(model.layers))

Number of layers in my model: 4


### Citation

[1] https://en.wikipedia.org/wiki/TensorFlow

[2] https://en.wikipedia.org/wiki/Keras

[3] https://stackoverflow.com/questions/41990250/what-is-cross-entropy

[4] http://scikit-learn.org/stable/modules/multiclass.html

[5] https://keras.io/utils/

[6] https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.permutation.html

[7] https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.split.html

[8] https://www.quora.com/In-AI-what-is-an-ML-model

[9] https://keras.io/layers/core/

[10] https://www.youtube.com/watch?v=-7scQpJT7uo

### End